<a href="https://colab.research.google.com/github/simecek/PseudoDNA_Generator/blob/master/experiments/Transcripts_MarkovModel_KMER3_v0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

In [1]:
!pip install git+https://github.com/ML-Bioinfo-CEITEC/rbp

  Cloning https://github.com/ML-Bioinfo-CEITEC/rbp to /tmp/pip-req-build-8k9v46ql
  Running command git clone -q https://github.com/ML-Bioinfo-CEITEC/rbp /tmp/pip-req-build-8k9v46ql
     |████████████████████████████████| 1.5MB 2.8MB/s 
     |████████████████████████████████| 256kB 17.3MB/s 
     |████████████████████████████████| 12.5MB 310kB/s 
     |████████████████████████████████| 71kB 9.0MB/s 
     |████████████████████████████████| 10.0MB 39.8MB/s 
  Created wheel for rbp: filename=rbp-0.1.3-cp36-none-any.whl size=11792 sha256=d7c23c316ec60d5769625ce2764e0bb5cf34fe6c00177c7580f5b6b413824372
  Stored in directory: /tmp/pip-ephem-wheel-cache-fxj0ilfg/wheels/2a/9c/79/c18ab902d7c45e2854bf635bd6fef7b7ee084b07dcf818ee97
  Created wheel for pybedtools: filename=pybedtools-0.8.1-cp36-cp36m-linux_x86_64.whl size=13606331 sha256=e63519b9e560d27421a14807882c86b8c31f6664addc43ac971e5bcd9060ca7d
  Stored in directory: /root/.cache/pip/wheels/6b/50/97/7d0e4f605d0d1578997f4bba3061869c2dee9f8cd

In [4]:
import pandas as pd
import numpy as np
from rbp.random.markov_model import MarkovModel
from tqdm import tqdm, notebook
notebook.tqdm().pandas()

In [2]:
# Mount to your Google Drive allowing lesson files will be saved to your Drive location
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


## Data

In [6]:
dt = pd.read_csv("/content/drive/My Drive/data/random/random_transcripts.csv")
dt.columns = ['id', 'chr', 'start', 'end', 'seq']
dt = dt[~dt.seq.str.contains("N")]   # just for sure

train = dt[dt.chr!="1"]
test = dt[dt.chr=="1"]

print(dt.shape, train.shape, test.shape)
dt.head()

(50000, 5) (45079, 5) (4921, 5)


,id,chr,start,end,seq
0,ENST00000443659,3,294,493,CTTCTGAAGAAAGAAATCGATAAGTTCAAAGACAAAAAAGAGGTAG...
1,ENST00000458477,12,647,846,CTGCACCCACTGCCAGGAGGAGATCGGATCCCGGAACTTCTTCGAG...
2,ENST00000610355,4,1042,1241,GTAGTCAATGTGAACCAATTACATTGGAACTCTGCATGAATTTGCC...
3,ENST00000536187,12,1055,1254,GGAGGTTTACTGGCGGGCCACGCACCACCCAGCCCCTGGCCCCGGA...
4,ENST00000561718,16,109,308,ATGAAGCCGAGAAGGCGCTGAAGCACATGGATGGAGGACAAATTGA...


## Markov Model

In [7]:
mm = MarkovModel(k=3)
mm.fit(train.seq)

In [8]:
def predict3(prev3):
    tmp = mm.kmer_pairs[mm.kmer_pairs.index.get_level_values('prev') == prev3].reset_index().drop(columns='prev').sort_values(0, ascending=False)
    return tmp.curr.iloc[0]

predict3("ACG")

'TGG'

In [9]:
next3 = {x: predict3(x) for x in mm.kmer_pairs.index.get_level_values('prev')}
next3['ACG']

'TGG'

In [10]:
def compare3(x, y):
    return int(x[0] == y[0]), int(x[1] == y[1]), int(x[2] == y[2])
compare3('GGG', 'AGG')

(0, 1, 1)

In [11]:
def stat3(s):
    sdf = pd.DataFrame({'prev': [s[i:i+3] for i in range(50,200-3,3)], 'next': [s[i:i+3] for i in range(53,200,3)]})
    sdf['nextpred'] = sdf.prev.apply(predict3)
    sdf[['c1', 'c2', 'c3']] = pd.DataFrame.from_records(sdf.apply(lambda row: compare3(row['next'], row['nextpred']), axis=1), columns=['c1', 'c2', 'c3'])
    return sdf[['c1', 'c2', 'c3']].mean(axis=0)

In [12]:
stats = test.seq.progress_apply(stat3)

In [14]:
stats.mean(axis=0)

c1    0.337197
c2    0.286934
c3    0.284872
dtype: float64